<a href="https://colab.research.google.com/github/midoriengineer/Charity-RPG-Technica-Hackathon-2022/blob/main/FINAL_VERSION_ERDC_Deep_Learning_Canonical_SMILES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOTE: TO RUN, CLICK RUNTIME, RESTART RUNTIME, THEN EITHER RUN EACH BLOCK IN ORDER OR CLICK RUN ALL**

This is the file that predicts toxicisity from chemical structure data, specifically Canonical SMILES strings. It used Deep Learning (Deep Neural Network). Currently, it predicts toxcisity as a binary value, to allow for comparison to the ERDC model. To run it, you will need the file called ERDC_Data_With_SMILES_VERSION-2.csv. This file is the ERDC data but with two additional columns labeled "SMILE" and 'CanonicalSMILE".

Import all necessary packages.

In [ ]:
pip install SmilesPE

In [ ]:
pip install DeepSMILES

In [ ]:
from SmilesPE.pretokenizer import atomwise_tokenizer
import pandas as pd
import deepsmiles
import tensorflow as tf
from tensorflow import keras
from random import shuffle
import numpy as np
from numpy import NaN
import math

This code block imports and tokenizes the SMILES strings from the data. This takes in a list of **Canonical** SMILES strings and first tokenizes them, then converts each token into a number representing a token. For example take the imaginary Canonical SMILES string "C[Na]C". This is what whoud heppen:
1. First the string will be tokenized as ['C', '[Na]', 'C']
2. Then each token will be converted to a token ID, which will take its place so ['C', '[Na]', 'C'] becomes [1, 2, 1] (assuming token IDs start at 1) we do not start as 0 as 0 is used as a padding to make all tokenized ID Canonical SMILES strings the same length.

In [ ]:
# Set to True to use DeepSMILES or False not to use Deep SMILES
# from experimentation, this does not have a significant effect
# on accuracy of the model despite our hopes
use_deep_smiles = True

# Setup deep smiles converter with default settings
converter = deepsmiles.Converter(rings=True, branches=True)

# Read in data from the ERDC dataset with Canonical SMILES data
df = pd.read_csv('ERDC_Data_With_SMILES_VERSION-2.csv')

# Get the canonical SMILES data as a list of strings
canonical_smiles = list(df['CanonicalSMILE'])

token_to_id = {} # hash set thet goes from a token to an ID for that token
id = 1 # the ID for each new token - will be incremented as new tokens are found
token_list_list = [] # the final list of token ID lists
for smile in canonical_smiles:

  # If canonical SMILE not available put in NAN and CONTINUE!
  if type(smile) == float:
    token_list_list.append(NaN)
    continue

  # Tokenize the canonical SMILE with either default tokenizing or DeepSMILES
  if use_deep_smiles:
    tokens = atomwise_tokenizer(str(converter.encode(smile)))
  else:
    tokens = atomwise_tokenizer(str(smile))

  # For each token, convert it to an ID for that specific token or if that token
  # is new, add it to the hash set and then convert it to a token ID
  for i in range(len(tokens)):
    token = tokens[i]
    if token in token_to_id:
      tokens[i] = token_to_id[token]
    else:
      token_to_id[token] = id
      id += 1
      tokens[i] = token_to_id[token]

  # Add thet tokenized ID list to the list of tokenized ID lists
  token_list_list.append(tokens)

highest_token_id = id - 1
print("Highes token ID:", highest_token_id)

len(df)

Highes token ID: 98


59117

Next, we pad the tokenized ID data with zeros so all the tokenized ID strings have the same length as the longest one.

In [ ]:
# Get the length of the longest tokenized ID string
longest_length = 0
for tokenized_list in token_list_list:

  # If float.NaN continue
  if type(tokenized_list) == float:
    continue

  length = len(tokenized_list)
  if length > longest_length:
    longest_length = length

print("Longest String Length:", longest_length)

# Now pad with zeros to length of longest_length
for token_list in token_list_list:

  # If float.NaN continue
  if type(token_list) == float:
    continue

  zeros_to_add = longest_length - len(token_list)

  for i in range(0, zeros_to_add):
    token_list.append(0)


# Add the tokenized canonical SMILESs data to the dataframe
df_to_add = pd.DataFrame({'Tokenized_Canonical_Smiles': token_list_list})
new_df = df.join(df_to_add)
new_df

Longest String Length: 205


,Id,Expected,ChemId,AssayId,PubChem_CID,MW,XLogP,TopoPSA,SCH.3,SCH.4,...,ATSm2,ATSm3,ATSm4,ATSm5,ECCEN,VAdjMat,ISOTOPE,SMILE,CanonicalSMILE,Tokenized_Canonical_Smiles
0,2971-36-0;1644,0,2971-36-0,0,76302,315.982463,4.592,40.46,0.0,0.0,...,26.519496,59.322530,41.038992,52.085432,290,5.321928,1,C(C(Cl)(Cl)Cl)(C1=CC=C(O)C=C1)C2=CC=C(O)C=C2,ClC(Cl)(Cl)C(C1=CC=C(O)C=C1)C2=CC=C(O)C=C2,"[1, 2, 1, 3, 1, 3, 2, 2, 4, 2, 2, 4, 2, 5, 3, ..."
1,693-54-9;2451,0,693-54-9,1,12741,156.151415,3.852,17.07,0.0,0.0,...,10.332092,10.664184,8.332092,7.332092,139,4.321928,1,C(CCCCC)CCC(C)=O,O=C(C)CCCCCCCC,"[5, 4, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, ..."
2,7173-51-5;1384,0,7173-51-5,2,23558,361.347528,9.912,0.00,0.0,0.0,...,22.664728,24.332364,22.332364,21.332364,1050326996,5.459432,1,C([N+](CCCCCCCCCC)(C)C)CCCCCCCCC.[Cl-],[Cl-].CCCCCCCCCC[N+](C)(C)CCCCCCCCCC,"[7, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 9, 2, 3, ..."
3,138261-41-3;16,0,138261-41-3,3,86418,255.052302,2.294,83.66,0.0,0.0,...,22.748130,32.352497,26.617141,23.522689,273,5.169925,1,C(N1\C(=N\N(=O)=O)\NCC1)C=2C=CC(Cl)=NC2,O=N(=O)N=C1NCCN1CC2=CN=C(Cl)C=C2,"[5, 4, 10, 4, 5, 3, 10, 4, 2, 10, 2, 2, 10, 11..."
4,7681-82-5;1856,0,7681-82-5,4,5238,149.894242,1.050,0.00,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0,0.000000,1,I[Na],[Na]I,"[12, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59112,528-29-0;28,0,528-29-0,43,10707,168.017107,1.806,86.28,0.0,0.0,...,14.546210,19.542034,19.681444,19.202946,98,4.584963,1,O(C)C=1C=C(C=CC1N)C2=CC(OC)=C(N)C=C2.Cl,Cl.O(C=1C=C(C=CC1N)C2=CC=C(N)C(OC)=C2)C,"[1, 8, 5, 2, 2, 4, 2, 2, 4, 2, 2, 4, 6, 10, 3,..."
59113,69-65-8;1387,0,69-65-8,86,5780,182.079038,-3.896,121.38,0.0,0.0,...,12.992552,17.320920,22.529081,17.090428,114,4.459432,1,ClC12C3C(C(Cl)(C1(Cl)Cl)C(Cl)=C2Cl)COS(=O)OC3,O=S1OCC2C(CO1)C3(Cl)C(Cl)=C(Cl)C2(Cl)C3(Cl)Cl,"[5, 4, 15, 5, 2, 2, 2, 2, 5, 20, 3, 3, 2, 1, 3..."
59114,57-97-6;1507,1,57-97-6,36,6001,256.125201,6.694,0.00,0.0,0.0,...,23.000000,34.000000,39.000000,35.000000,308,5.523562,1,OC1=C(C)C=CC=C1C,OC=1C(=CC=CC1C)C,"[5, 2, 2, 4, 2, 2, 4, 2, 2, 4, 6, 2, 3, 3, 3, ..."
59115,92-87-5;38,1,92-87-5,29,7111,184.100048,2.328,52.04,0.0,0.0,...,15.332364,20.664728,18.664728,12.332364,200,4.906891,1,C(F)(F)(F)C1=C(C#N)C=CC(NC(C(CS(=O)(=O)C2=CC=C...,N#CC1=CC=C(C=C1C(F)(F)F)NC(=O)C(O)(C)CS(=O)(=O...,"[10, 17, 2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 6, 2, 1..."


Prepare our training and testing data by retrieving the input and expected output. Here is where we down sample to give us an equal number of both toxic and nom-toxic records. We end up with about 16k records for both training and testing, which is significantly lower than what we need for an acurate model, but we wer un-able to find more reputable data in large enough quantities that was free and un-classified.

In [ ]:
# Prints the number of NaNs - should be 0
print('NaNs:', new_df.isna().sum().sum())

# If the entire 59,117 record file was not uploaded, raise an exception
expected_file_length = 59117
if len(new_df) != expected_file_length:
  raise Exception('ERROR: FILE DID NOT UPLOAD IN FULL')

# Shuffle rows
new_df = new_df.sample(frac=1)

output = list(new_df['Expected']) # pandas.core.series.Series of numpy.int64
input = list(new_df['Tokenized_Canonical_Smiles']) # pandas.core.series.Series of list

new_output_1s = []
new_input_1s = []

new_output_0s = []
new_input_0s = []
for i in range(0, len(input)):

  inp = input[i]
  outp = output[i]

  if (outp == 1) and (len(new_output_1s) < 8200) :
    new_output_1s.append(outp)
    new_input_1s.append(inp)
  elif (outp == 0) and (len(new_output_0s) < 8200):
    new_output_0s.append(outp)
    new_input_0s.append(inp)

print('Length of 1s:', len(new_output_1s))
print('Length of 0s:', len(new_output_0s))

input = new_input_1s + new_input_0s
output = new_output_1s + new_output_0s

# Split data into testing and training. The vairable "percent" represents the
# precent of the data used. It can me any value from 0 to 100 (inclusive)
percent = 90

split_point = math.floor((percent/100) * len(input))

input_training = input[0:split_point]
output_training = output[0:split_point]

input_testing = input[split_point:]
output_testing = output[split_point:]

new_df

NaNs: 0
Length of 1s: 8200
Length of 0s: 8200


,Id,Expected,ChemId,AssayId,PubChem_CID,MW,XLogP,TopoPSA,SCH.3,SCH.4,...,ATSm2,ATSm3,ATSm4,ATSm5,ECCEN,VAdjMat,ISOTOPE,SMILE,CanonicalSMILE,Tokenized_Canonical_Smiles
18108,147-24-0;1856,1,147-24-0,4,3100,255.162314,3.158,12.47,0.000000,0.00000,...,21.162730,26.162458,25.881830,25.158734,285,5.321928,1,C(OCCN(C)C)(C1=CC=CC=C1)C2=CC=CC=C2.Cl,Cl.O(CCN(C)C)C(C=1C=CC=CC1)C=2C=CC=CC2,"[1, 8, 5, 2, 2, 10, 2, 3, 2, 3, 3, 3, 3, 2, 2,..."
34394,NOCAS_48516;1507,0,NOCAS_48516,36,23700188,360.995164,4.033,59.59,0.000000,0.00000,...,34.927869,44.170176,45.405950,49.628223,-1244640302,5.523562,1,S(N(C)C)(=O)(=O)N1C(=C(Cl)N=C1C#N)C2=CC=C(C)C=C2,N#CC1=NC(Cl)=C(C=2C=CC(=CC2)C)N1S(=O)(=O)N(C)C,"[10, 17, 2, 2, 4, 10, 2, 1, 3, 4, 2, 2, 2, 4, ..."
6380,101-05-3;1654,0,101-05-3,67,7541,273.957979,2.883,50.70,0.000000,0.00000,...,24.184767,38.805013,36.648563,39.576588,217,5.087463,1,N(C=1N=C(Cl)N=C(Cl)N1)C2=C(Cl)C=CC=C2,ClC=1N=C(Cl)N=C(N1)NC=2C=CC=CC2Cl,"[1, 2, 10, 4, 2, 1, 3, 10, 4, 2, 10, 4, 6, 3, ..."
16323,10453-86-8;2443,0,10453-86-8,114,5053,338.188195,4.971,39.44,0.166667,0.54741,...,28.660460,38.767021,33.988828,35.095389,622,5.754888,1,C(=C(C)C)C1C(C(OCC=2C=C(CC3=CC=CC=C3)OC2)=O)C1...,O=C(OCC1=COC(=C1)CC=2C=CC=CC2)C3C(C=C(C)C)C3(C)C,"[5, 4, 2, 5, 2, 2, 4, 2, 5, 2, 4, 2, 11, 3, 2,..."
55319,552-30-7;2451,0,552-30-7,1,11089,192.005873,1.138,80.67,0.000000,0.00000,...,16.992552,26.315959,24.985104,19.095389,158,4.906891,1,C(C)(C)(C1=CC=C(O)C=C1)C2=CC=CC=C2,OC1=CC=C(C=C1)C(C=2C=CC=CC2)(C)C,"[5, 2, 4, 2, 2, 4, 2, 2, 4, 2, 6, 3, 3, 2, 2, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20776,2425-77-6;2,0,2425-77-6,88,95337,242.260966,7.407,20.23,0.000000,0.00000,...,16.332092,16.332092,16.664184,15.664184,330,5.000000,1,C(CCCCCCCC)(CCCCCC)CO,OCC(CCCCCC)CCCCCCCC,"[5, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, ..."
14099,13194-48-4;1508,1,13194-48-4,109,3289,242.056408,2.627,86.71,0.000000,0.00000,...,32.312061,39.536001,30.415690,25.497280,138,4.584963,1,P(SCCC)(SCCC)(OCC)=O,O=P(OCC)(SCCC)SCCC,"[5, 4, 14, 5, 2, 2, 3, 3, 3, 15, 2, 2, 2, 3, 3..."
54450,NOCAS_48507;1384,0,NOCAS_48507,2,60196436,412.111375,3.002,87.58,0.000000,0.00000,...,32.987576,50.808169,52.137150,46.782861,-2129542202,5.857981,1,O[C@@]12[C@@]([C@@H](O)[C@]3(C(=C1O)C(=O)C=4C(...,O=C(N)C=1C(=O)C2(O)C(O)=C3C(=O)C=4C(O)=CC=CC4C...,"[5, 4, 2, 10, 3, 2, 2, 4, 5, 3, 2, 5, 3, 2, 5,..."
11517,87820-88-0;1637,0,87820-88-0,11,91746,329.199094,4.516,55.40,0.000000,0.00000,...,26.715919,37.491098,43.321464,47.202584,473,5.643856,1,CC1=C(C(C)=CC(C)=C1)C2CC(=O)C(C(=NOCC)CC)=C(O)C2,O=C1C(=C(O)CC(C=2C(=CC(=CC2C)C)C)C1)C(=NOCC)CC,"[5, 4, 2, 2, 4, 2, 5, 3, 2, 2, 2, 2, 4, 2, 2, ..."


Setup Deep Learning model and train with training data. Here you can change the number of layers, the number of units (number of neurons) and the activation function. You can also change the loss function.

NOTE: The first layer must have a number of neurons (units) equal to the length of the input lists, here 194. The last layer must have a single neuron (unit) to predict toxcisity on a binary spectrum for comparison to the ERDC model.

In [ ]:
# Get our Neural Network Model
model = tf.keras.models.Sequential()

# Add layers to the model
model.add(tf.keras.layers.Dense(units=longest_length, activation='relu')) # MUST have number of neurons (units) equal to the length of the input lists
model.add(tf.keras.layers.Dense(units=200, activation='relu'))
model.add(tf.keras.layers.Dense(units=175, activation='relu'))
model.add(tf.keras.layers.Dense(units=150, activation='relu'))
model.add(tf.keras.layers.Dense(units=125, activation='relu'))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=75, activation='relu'))
model.add(tf.keras.layers.Dense(units=25, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='relu'))
model.add(tf.keras.layers.Dense(units=5, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # MUST have 1 neuron (unit) to predict either toxic or not

# Compile/create the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

# Train the model with the training data
model.fit(input_training, output_training, epochs=100)

Epoch 1/100
462/462 [==============================] - 6s 7ms/step - loss: 0.6870 - accuracy: 0.5535
Epoch 2/100
462/462 [==============================] - 2s 5ms/step - loss: 0.6844 - accuracy: 0.5556
Epoch 3/100
462/462 [==============================] - 3s 5ms/step - loss: 0.6823 - accuracy: 0.5591
Epoch 4/100
462/462 [==============================] - 2s 5ms/step - loss: 0.6802 - accuracy: 0.5642
Epoch 5/100
462/462 [==============================] - 3s 7ms/step - loss: 0.6779 - accuracy: 0.5698
Epoch 6/100
462/462 [==============================] - 3s 5ms/step - loss: 0.6764 - accuracy: 0.5696
Epoch 7/100
462/462 [==============================] - 3s 5ms/step - loss: 0.6755 - accuracy: 0.5696
Epoch 8/100
462/462 [==============================] - 3s 5ms/step - loss: 0.6726 - accuracy: 0.5720
Epoch 9/100
462/462 [==============================] - 3s 7ms/step - loss: 0.6712 - accuracy: 0.5806
Epoch 10/100
462/462 [==============================] - 3s 6ms/step - loss: 0.6694 - accura

In [ ]:
model.evaluate(input_testing, output_testing)

52/52 [==============================] - 0s 2ms/step - loss: 1.0135 - accuracy: 0.4232


[1.0134615898132324, 0.4231707453727722]

Predicts binary toxicology based on a Canonical SMILES string.

In [ ]:
canonical_smiles_str = 'O=C(C)C'

# If deep smiles is used, convert to deep smiles
if use_deep_smiles:
  canonical_smiles_str = converter.encode(canonical_smiles_str)

# Tokenize canonical smiles string
tokenized_str = atomwise_tokenizer(canonical_smiles_str)

# Convert tokens to id numbers
tokenized_id_str = []
new_tokens = False
for token in tokenized_str:

  if token in token_to_id:
    tokenized_id_str.append(token_to_id[token])
  else:
    token_to_id[token] = id
    id += 1
    new_tokens = True
    tokenized_id_str.append(token_to_id[token])

print(tokenized_id_str)

# Padwith zeros to maximum length
if len(tokenized_id_str) > longest_length:
  raise Exception('The provided Canonical SMILES string is too long. The maximum length is ' + str(longest_length))
print(longest_length)
for i in range(longest_length - len(tokenized_id_str)):
  tokenized_id_str.append(0)

probability = model.predict([tokenized_id_str])[0][0]

print('According to the model there is a ' + str(probability * 100) + '% chance that the molecule is toxic')

p = list(model.predict(input_testing))

print("Max probability of testing set:", max(p))
print("Min probability of testing set:", min(p))
len(p)

[5, 4, 2, 2, 3, 2]
205
1/1 [==============================] - 0s 150ms/step
According to the model there is a 25.59545934200287% chance that the molecule is toxic
52/52 [==============================] - 0s 3ms/step
Max probability of testing set: [1.]
Min probability of testing set: [0.0004602]


1640